In [93]:
import cv2
import tempfile
import numpy as np
import torch
import torch.nn.functional as F
import os
from PIL import Image
import time
import requests
import json
import base64
import threading
from IPython.display import Audio

In [94]:
def sendToBackend(frame, audio = None):
    try:
        # Save current frame to disk
        cv2.imwrite("frame.jpg", frame)

        # Create an empty audio file (1 second of silence if needed)
        empty_audio_path = "input.mp3"
        if not os.path.exists(empty_audio_path):
            with open(empty_audio_path, "wb") as f:
                f.write(b"")

        with open("frame.jpg", "rb") as img, open("input.mp3", "rb") as audio:
            files = {
                "image": ("frame.jpg", img, "image/jpeg"),
                "audio": ("input.mp3", audio, "audio/mpeg")
            }

            response = requests.post("http://localhost:8000/process/", files=files)
            #response = requests.post("https://8804-2600-1017-a410-36b8-2357-52be-1318-959b.ngrok-free.app/process/", files=files)
            
            if response.status_code == 200: #If the request was successful
                print("Frame sent successfully!")
                return response.json()
            else:
                print(f"Failed: {response.status_code} - {response.text}")
                return None
    except Exception as e:
        print(f"Error sending frame: {e}")

In [95]:
frame = cv2.imread("1000268201.jpg")
response = sendToBackend(frame)

Frame sent successfully!


In [96]:
print("Response from backend:", response)

Response from backend: {'caption': "You are in an outdoor rustic setting. In front of you, a young girl in a pink dress is reaching up to a wooden structure with a lattice-patterned window and a woman's head carving. There's a green bucket on the ground nearby. In the background, there are pink-flowery plants and a wooden wall.", 'audio_base64': '...'}


In [97]:
response['caption']

"You are in an outdoor rustic setting. In front of you, a young girl in a pink dress is reaching up to a wooden structure with a lattice-patterned window and a woman's head carving. There's a green bucket on the ground nearby. In the background, there are pink-flowery plants and a wooden wall."

In [98]:
Audio(base64.b64decode(response["audio_base64"]))